# Camera Calibration


The 3D plant geometry was reconstructed from multiple images, each of which was a projection of the 3D plant into a plane (i.e. 2D image). However, the position and orientation of the plane in a global reference frame is unknown. Therefore, the Phenomenal pipeline includes an optimization algorithm (Salvi et al., 2002) for multi-view camera calibration, a classical procedure in computer vision (reference). This algorithm minimized the distance between a reference frame, a chessboard in our case, and its multi-view projections on its images by modifying the intrinsic and extrinsic parameters of a model camera. The position and orientation of the model camera for each view was computed globally since the intrinsic parameters of the camera (i.e. focal length, aspect ratio, skew) were fixed during an experiment and the camera calibration before that experiment. The extrinsic parameters (i.e. translation and rotation matrices) were computed independently for each view used for 3D reconstruction. In our system, the camera is fixed, but the target object (i.e. the plant) is rotated around an unknown axis of a given angle. The position of the rotation axis (X,Y) and the height of the camera is used as initial parameters of the optimisation algorithm.

# Plant image segmentation

Before the stage of 3D-reconstruction, Phenomenal pipeline separates the background pixels of each original image (the image cabin) from the foreground (the target plant) to produce a binary image. This automatic separation was achieved by combining two threshold algorithms. First, a mean-shift threshold algorithm (Comaniciu and Meer, 2002) was applied to separate the fixed part (the background) from the moving part (the plant) of the images obtained from the same plant but from different views :

$$min_{RGB}(\frac{image_{k}}{\overline{image}}) <= 0.70$$

* $image_{k}$: k-th RGB image of the serie
* $image$: mean RGB image from the different views  (see Figure XX.A)
* $min_{RGB}$ : minimum value of the RGB tuple for each pixel

Then, if some parts of the plant are located on the axis of rotation and are not moving (e.g. the stem), they are removed by the mean/shift threshold algorithm. To recover these pixels, we use an HSV threshold algorithm (Sural et al., 2002). To do so, images are converted into HSV color space and pixels contained in the pre-defined bound values (i.e. green, brown) are kept:

$$HSV <= lowerboundHSVimage <= HSVupperbound$$
$$\equiv$$
$$H_{lowerb} <= H_{image} <= H_{upperb} \land S_{lowerb} <= S_{image} <= S_{upperb} \land V_{lowerb} <= V_{image} <= V_{upperb} $$



Finally, a media blur (Huang et al, 1979) is applied to erase remaining noise. 


In [1]:
# Display Figure 
from alinea.phenomenal.display import *
from alinea.phenomenal.data_access import *
from alinea.phenomenal.binarization import *


raw_images = plant_1_images()
print raw_images

defaultdict(<function <lambda> at 0x0000000010862CF8>, {'top': defaultdict(None, {0: array([[[ 27,  48,  91],
        [ 27,  48,  93],
        [ 26,  51,  95],
        ..., 
        [ 23,  63, 135],
        [ 23,  72, 139],
        [ 18,  64, 119]],

       [[ 27,  51,  94],
        [ 27,  51,  95],
        [ 27,  51,  92],
        ..., 
        [ 23,  62, 134],
        [ 23,  72, 139],
        [ 20,  72, 121]],

       [[ 27,  50,  95],
        [ 27,  50,  92],
        [ 27,  50,  89],
        ..., 
        [ 16,  59, 133],
        [ 17,  59, 134],
        [ 13,  59, 116]],

       ..., 
       [[102, 128, 126],
        [102, 129, 128],
        [102, 132, 131],
        ..., 
        [116, 155, 149],
        [116, 155, 146],
        [116, 155, 146]],

       [[100, 128, 127],
        [ 98, 124, 126],
        [100, 129, 130],
        ..., 
        [116, 154, 143],
        [116, 154, 140],
        [116, 154, 143]],

       [[ 98, 128, 127],
        [ 96, 124, 126],
        [ 98, 129, 130